# Prueba Coink - Cientifico de Datos
En este Jupyter Notebook se desarrollan los 3 puntos de la prueba Coink para el cargo Cientifico de Datos con fecha de entrega domingo 19 de marzo 2023. Además del enunciado, para la realizar la prueba se necesitan dos datasets que estan contenidos en el folder resources. A continuación se cargan las librerias usadas en este notebook.

In [1]:
import pandas as pd

### Punto 1
En este punto piden evaluar que tan buenos son los usuarios de Coink y calificar a los usuarios contenidos en la muestra depositos_oink.csv. Lo primero es cargar los datos para hacer una primera exploración

In [14]:
# Lee el .csv de resources. La primera columna (Unnamed: 0) son indices
depositos_oink_df = pd.read_csv("resources\depositos_oinks.csv", index_col = 'Unnamed: 0')

# Cast columnas que incluyen fechas a datetime
depositos_oink_df[['operation_date', 'user_createddate']] = \
    depositos_oink_df[['operation_date', 'user_createddate']].apply(lambda x: pd.to_datetime(x))

depositos_oink_df.head(3)

,user_id,operation_value,operation_date,maplocation_name,user_createddate
0,0e52d550-ae23-407a-9b1f-6e5fb1f066ab,273850.0,2022-01-14 13:07:56,CC Plaza de las Américas - Plaza Mariposa,2022-01-09 19:23:18.332689
1,975ed41e-d891-4c23-aeba-06363019d8e3,900.0,2022-02-06 13:11:25,CC Plaza de las Américas - Plaza Mariposa,2022-01-10 15:19:29.419075
2,f9043545-dfc2-402f-a8c7-0a61f21cb719,1800.0,2022-02-23 14:01:56,Universidad de los Andes - ML Piso 5,2021-08-25 12:44:48.524941


Después de un breve análisis, la métrica para calificar usuarios (user_score) propuesta se describe a continuación.

$$user score = 0.5C_{monto}+0.5C_{recurrencia}$$
$user score$ es la suma de dos coeficientes que miden aspectos distintos en la importancia del usuario. Por un lado $C_{recurrencia}$ cuantifica la recurrencia con la que la persona hace depositos. Adicional, $C_{monto}$ cuantifica el tamaño de los depositos.

Específicamente, $C_{recurrencia}$ se define como el tiempo promedio entre cada operación. La idea es que entre menor sea el tiempo promedio entre transacciones el usuario puntua más alto. Adicionalmente, se divide dicho promedio entre el tiempo de creación de la cuenta más vieja de la muestra. La división se hace para que $C_{recurrencia}$ quede en numeros adimensionales y se prioricen las personas más antiguas en caso de tiempos promedios entre transacciones similares.

Para estimar $C_{monto}$, simplemente se divide el monto total transferido por el usuario entre el mayor monto total entre todos los usuarios. Esto se hace para que el coeficiente quede en valores adimensionales y se le asigne un puntaje más alto a los que tienen más dinero transferido.

Por último, el 0.5 al lado de cada coeficiente es el peso. Para esta prueba se le da el mismo peso a ambos. Adicional, esto nos garantiza que $0<user score<=1$, siempre y cuando la suma de los pesos sea 1, ya que los coeficientes ya cumplen esta desigualdad

##### Calculo $C_{recurrencia}$
A continuación se calcula el $C_{recurrencia}$ de cada usuario. Este está definido como:
$$
C_{recurrencia}=1-\frac{\hat{\Delta t}}{Tiempo\ de\ creación\ usuario}
$$
Donde $\hat{\Delta t}$ es el tiempo promedio entre transacciones que se puede calcular como
$$
\hat{\Delta t}=\frac{(t_1-t_2)+(t_2-t_3)+...+(t_{n-1}-t_n)}{número\ de\ transacciones}=\frac{t_1-t_n}{número\ de\ transacciones}
$$
Donde $t_i$ es la fecha de la transacción $i$

In [18]:
print(len(depositos_oink_df['user_id'].unique()))
print(len(depositos_oink_df['user_id']))

1656
4345
